In [ ]:
from matplotlib import rcParams, rc
rcParams.update({'figure.autolayout': True})

import csv
import torch
import os
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np
import math
import pandas as pd

from Tarjet import *
from Phisicsparams import *
from utils import *
from MTMM import *
from tqdm import tqdm

from multiresglonet import GLOnet
from material_database import MatDatabase


In [ ]:
from typing import TypeVarTuple


params = Params()
params.thickness_sup = 0.2
params.N_layers = 10

#--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
params.materials = ['Al2O3','TiO2', 'SiO2']
params.user_define = True
if params.user_define:
  params.n_min = 1.09
  params.n_max = 2.6
  params.M_discretion_n = 200
  params.M_materials = params.M_discretion_n
  params.n_database = torch.tensor(np.array([np.linspace(params.n_min,params.n_max,params.M_discretion_n)]))
else:
  pass # definirlo en otro lado
#---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

params.alpha_sup =  5
params.numIter = 10
params.sigma = 0.035
params.batch_size = 50
#---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
params.net = 'Res'
params.res_layers = 16                                                                             # Cantidad de bloques Residuales del bloque ResNet
params.res_dim = 256                                                                               # Cantidad de neuronas en la capa de entrada al bloque ResNet
params.noise_dim = 26                                                                              # Dimension de la Capa de entrada
params.lr = 0.05                                                                                   # Tasa de aprendizaje del optimizador Adam (learning rate)
params.beta1 = 0.9                                                                                 # Coeficiente de decaimiento para el momento del primer orden del optimizador Adam
params.beta2 = 0.99                                                                                # Coeficiente de decaimiento para el momento del segundo orden del optimizador Adam
params.weight_decay = 0.001                                                                        # Termino de decaimiento del peso para regularizar los pesos del generador durante la optimizacion
params.step_size = 40000                                                                           # Numero de epicas despues de las cuales se reduce la tasa de aprendizaje
params.gamma = 0.5                                                                                 # El factor de reduccion para la tasa de aprendizaje. Despues de cada step_size epocas, la tasa de aprendizaje se multiplica por gamma


In [ ]:
params.condiciones = 3
physicsparams = PhysicsParams(params.condiciones, user_define=True)

physicsparams.n_bot_1 = 1.4
physicsparams.n_top_1 = 1
physicsparams.k_1 = 370
physicsparams.k_values.append(physicsparams.k_1)
physicsparams.theta_1 = 45
physicsparams.lambda_min_1 = 380
physicsparams.lambda_max_1 = 750
physicsparams.pol_1 = "both"

physicsparams.n_bot_2 = 1
physicsparams.n_top_2 = 1
physicsparams.k_2 = 190
physicsparams.k_values.append(physicsparams.k_2)
physicsparams.theta_2 = 45
physicsparams.lambda_min_2 = 380
physicsparams.lambda_max_2 = 570
physicsparams.pol_2 = "TE"

physicsparams.n_bot_3 = 1
physicsparams.n_top_3 = 1
physicsparams.k_3 = 115
physicsparams.k_values.append(physicsparams.k_3)
physicsparams.theta_3 = 0
physicsparams.lambda_min_3 = 380
physicsparams.lambda_max_3 = 495
physicsparams.pol_3 = "TE"

physicsparams.generate_physics_params()
# physicsparams.view_attributes()

tarjet = Tarjet(params.condiciones, physicsparams.k_values, params.user_define)
tarjet.configure_targets()



In [ ]:
if not params.user_define:
    params.matdatabase =  MatDatabase(params.materials)
    params.n_database = params.matdatabase.interp_wv(2 * math.pi/physicsparams.ks, params.materials, True)
    params.M_materials =  params.n_database.size(0)

In [ ]:
for seed in range(1):                                                # Generacion del comando for para las semillas (0, ->(cantidad de semillas aleatorias), 1)
  params.seed = seed

  torch.manual_seed(seed)
  glonet = GLOnet(params, physicsparams, tarjet)
  glonet.train()
  glonet.viz_training()
  print(f"iteration{seed + 1 }")


In [ ]:
plt.figure(figsize = (20, 5))
plt.subplot(131)
plt.plot(glonet.loss_training)
plt.ylabel('Loss', fontsize=18)
plt.xlabel('Iterations', fontsize=18)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)

In [ ]:
with torch.no_grad():
  glonet.generator.eval()
  (thicknesses, ref_index_mat, result_mat) = glonet.evaluate(100, kvector = params.k_test,  inc_angles = params.theta_test, grayscale=False)
  FoM = torch.pow(reflection_mat - params.target_reflection.cpu(), 2).mean(dim=[1, 2, 3]) # type: ignore
  _, indices = torch.sort(FoM)
  opt_idx = indices[0]

reflection_mat_opt = reflection_mat[opt_idx].detach().cpu() # type: ignore
ref_idx_opt_mat =  ref_index_mat[opt_idx].detach().cpu()
thickness_opt = thicknesses[opt_idx].detach().cpu()

mat_opt = [params.materials[result_mat[opt_idx, i]] for i in range(result_mat.size(1))]

In [ ]:
with torch.no_grad():
  num_devices = 6
  params.k_test = 2 * math.pi / torch.linspace(0.3, 2.5, physicsparams.k_values[0])
  params.theta_test = torch.linspace(0, math.pi/2.25, physicsparams.k_values[0])

(thicknesses, ref_index_mat, result_mat) = glonet.evaluate(100, kvector = params.k_test,  inc_angles = params.theta_test, grayscale=False)
(optimal_thicknesses, optimal_ref_idx, optimal_reflections) = glonet.optimizer_evaluate(num_devices, params.k_test, params.theta_test )

FoM = torch.pow(optimal_reflections["reflexion_1"] - tarjet.tarjets["tarjet_1"], 2).mean(dim=[1, 2, 3])
_, indices = torch.sort(FoM)
opt_idx = indices[0]

mat_opt = [params.materials[result_mat[opt_idx, i]] for i in range(result_mat.size(1))]
print(mat_opt)
print(optimal_reflections['reflexion_1'].shape)
print(optimal_thicknesses)
print(optimal_ref_idx)


plt.plot(2*math.pi / physicsparams.k_1 * 1000, optimal_reflections['reflexion_1'][:,0,0].detach().numpy(), "o-", color = "red", label = "Reflexion")
plt.plot(2*math.pi / physicsparams.k_1 * 1000,tarjet.tarjets["tarjet_1"].view(-1), "o-", color = "green", label =  "Tarjet")
plt.xlabel("Wavelength (nm)", fontsize = 15)
plt.ylabel("Reflection", fontsize = 15)
plt.legend(loc= "best")


In [ ]:
# Lista de espesores
espesores = optimal_thicknesses["reflexion_1"].detach().numpy()
materials = mat_opt


colores =  []
for i in range(12):
  if materials[i] == 'TiO2':
    colores.append('orange')
  elif materials[i] == 'SiO2':
    colores.append("Violet")
  elif materials[i] == 'Al2O3':
    colores.append('Blue')



# Crear la figura y el eje
fig, ax = plt.subplots()

# Calcular la suma acumulada de los espesores
espesores_acumulados = np.cumsum(espesores)

# Graficar las barras con colores diferentes
for i in range(len(espesores)):
    ax.bar(1, espesores[i], bottom=espesores_acumulados[i] - espesores[i], color=colores[i])

# Etiquetas y título

ax.set_ylabel('Espesor (m)', fontsize = 15)


# Mostrar la gráfica
plt.tight_layout()
plt.show()

In [ ]:
with torch.no_grad():
  params.k_test = 2 * math.pi / torch.linspace(0.3, 1.5, 100)
  params.theta_test = torch.linspace(0, math.pi/2.25, 80)
  glonet.generator.eval()

  z = glonet.sample_z(200)
  thicknesses, refractive_indices, P = glonet.generator(z, glonet.alpha)


  reflex = MTMM_solver(condiciones, thicknesses, refractive_indices, physicsparams) # type: ignore
  reflection_1 = TMM_solver(thicknesses, refractive_indices, physicsparams.n_bot_1, physicsparams.n_top_1 ,physicsparams.k_1, physicsparams.theta_1, physicsparams.pol_1)
  # reflection_2 = TMM_solver(thicknesses, refractive_indices, physicsparams.n_bot_2, physicsparams.n_top_2 ,params.k_test, params.theta_test, physicsparams.pol_2)

In [ ]:
FoM = torch.pow(reflection_1 - tarjet.tarjets["tarjet_1"], 2).mean(dim=[1, 2, 3])
_, indices = torch.sort(FoM)

opt_idx = indices[0]
ref_idx_opt = refractive_indices[opt_idx].detach().cpu()
thickness_opt = thicknesses[opt_idx].detach().cpu()
reflection_opt = reflection_1[opt_idx].detach().cpu()

In [ ]:
print(ref_idx_opt)
print(thickness_opt)


In [ ]:

best_thicknesses =  torch.tensor([[ 0.562, 0.873 , 1.39]])
best_ref_indx = torch.tensor([[[2.6],[1.66],[1.15]]])


best_reflexion =  TMM_solver(best_thicknesses, best_ref_indx, physicsparams.n_bot_1, physicsparams.n_top_1 ,physicsparams.k_1, physicsparams.theta_1, physicsparams.pol_1 )
print(best_reflexion.shape)
best_reflexion[0,:,0,0]

In [ ]:
reflection_opt[:,0,0].view(-1).numpy()
print(reflection_opt[:,0,0].view(-1).numpy())
print(reflection_opt.shape)
plt.figure(figsize=(7,3))
plt.plot( 2*math.pi / physicsparams.k_1 *1000   ,reflection_opt[:,0,0].view(-1).numpy(), "o-", label = "reflexion", color = "red")
plt.ylabel("Reflecitivty", fontsize= 15)
plt.xlabel("Wavelength (nm)", fontsize = 15)

# plt.plot(np.arange(len(best_reflexion[0,:,0,0])),best_reflexion[0,:,0,0], "o", label = "best_reflexion")
plt.legend()